# FAA Bird Strike EDA

Nesta seção, vamos pegar as descobertas resumidas do curso Anaconda [_Análise Exploratória de Dados com Python_](https://learning.anaconda.cloud/exploratory-data-analysis-eda-with-python) e reuni-las para contar uma história.

Digamos que um sindicato de pilotos nos pediu para realizar um estudo independente sobre os fatores que causam colisões com pássaros. Existem outros relatórios por aí, mas eles querem fazer os seus próprios e ver se chegamos a conclusões semelhantes, visto que os danos causados ​​por colisões com pássaros continuam sendo um problema.

Queremos incluir apenas dados recentes de 2015 em diante. Consideramos que há pouco valor em analisar dados mais antigos, anteriores a esse ano. Podemos argumentar que a natureza das colisões com pássaros nunca muda, mas o ambiente certamente muda. Horários e aeroportos aumentam e diminuem, os padrões climáticos mudam e diferentes companhias aéreas surgem e desaparecem. Até mesmo a [própria FAA](https://wildlife.faa.gov/home) afirma:

> A expansão das populações de animais selvagens, o aumento no número de movimentos de aeronaves, a tendência para aeronaves mais rápidas e silenciosas e a conscientização da comunidade da aviação contribuíram para o aumento observado nos registros de colisões com animais selvagens.

## Conjunto de Dados de Colisões com Aves

Colisões com aves em aeronaves, conforme relatado pela Administração Federal de Aviação (FAA) nos Estados Unidos. Uma colisão com aves ocorre quando uma ave colide com uma aeronave, e os danos podem ser graves. A cada ano, ocorrem em média 13.000 colisões com aves somente nos Estados Unidos, custando à indústria da aviação US$ 400 milhões em danos. Embora a maioria das colisões com aves seja leve, algumas podem ser perigosas e fatais.

<a title="Greg L, CC BY 2.0 &lt;https://creativecommons.org/licenses/by/2.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:US_Airways_Flight_1549_(N106US)_after_crashing_into_the_Hudson_River_(crop_2).jpg"><img width="512" alt="Voo 1549 da US Airways (N106US) após cair no Rio Hudson (corte 2)" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/8f/US_Airways_Flight_1549_%28N106US%29_after_crashing_into_the_Hudson_River_%28crop_2%29.jpg/512px-US_Airways_Flight_1549_%28N106US%29_after_crashing_into_the_Hudson_River_%28crop_2%29.jpg?20200816213116"></a>

**Em 2009, o voo 1549 da US Airways sofreu uma grave colisão com pássaros, resultando em um pouso de emergência no Rio Hudson.**
*Greg L, CC BY 2.0, via Wikimedia Commons*

Abaixo, importaremos os dados de colisão com pássaros que limpamos de outro curso da Anaconda.

In [ ]:
import pandas as pd 

url = r"https://github.com/thomasnield/anaconda_python_eda/raw/public/birdstrike_section2.csv"
df = pd.read_csv(url, index_col='INDEX_NR', parse_dates=["INCIDENT_DATE"])

with pd.option_context('display.max_columns', None):
  display(df)

Também faremos algumas conversões de tipos de dados.

In [ ]:
# Transforma PHASE_OF_FLIGHT em uma categoria
phase_of_flt = pd.CategoricalDtype(categories=['Parked', 'Taxi','Take-off Run', 'Approach', 'Departure', 'Climb', 'En Route',
                                               'Descent', 'Landing Roll', 'Arrival', 'Local'])

df["PHASE_OF_FLIGHT"] = df["PHASE_OF_FLIGHT"].astype(phase_of_flt)

# Transforma TIME em tipo timedelta
df["TIME"] = pd.to_timedelta(df["TIME"])

Cada registro é relatado por um piloto à FAA. Uma coisa é se o piloto trabalha para uma grande companhia aérea como a Delta ou a Southwest Airlines, e eles são bem treinados em procedimentos. Eles têm poucos motivos para deixar de registrar um relatório de colisão com pássaros, além do tempo que isso leva. Mas um piloto independente que possui sua própria aeronave pequena pode ser uma incógnita. Se o dano for pequeno ou inexistente, talvez ele simplesmente ignore e não o relate. Se ele atingir uma espécie protegida como a águia-americana, ele pode ficar ainda menos inclinado, pois não tem certeza das consequências.

Advertimos nosso público que o autorrelato sempre carrega um viés, pois nem todo mundo vai fazer isso. Isso pode distorcer os resultados de maneiras que não refletem a realidade. Por exemplo, se os dados mostram que grandes companhias aéreas são muito mais propensas a colisões com pássaros do que aeronaves da aviação geral, isso pode ser devido ao fato de as companhias aéreas serem melhores em reportar, e não porque os pássaros colidem mais com aviões.

Devemos desafiar nosso público a não se prender muito apenas ao que os dados dizem. Eles também devem perguntar de onde vieram. O que poderia enviesá-los? Formular essas perguntas ajudará a formular conclusões mais inteligentes.

Dito isso, vamos passar para algumas descobertas bem reais.

![](https://y.yarn.co/18f66e28-0f40-4db7-b05f-3529759e9708_text.gif)

*Fonte: Paramount Pictures* 

## Altura e Fase de Voo

Uma das primeiras coisas que notamos em nossa análise é que as colisões com pássaros definitivamente se concentram em altitudes mais baixas, especialmente abaixo de 1.000 pés.

In [ ]:
df["HEIGHT"].hist(bins=30)

Se observarmos a fase do voo, surpreendentemente vemos que a "aproximação" e a "rolagem de pouso" têm mais impactos do que a decolagem ou a subida.

In [ ]:
df["PHASE_OF_FLIGHT"].value_counts().plot.bar()

Como estamos falando com pilotos, não precisamos definir cada uma dessas fases do voo para eles, pois provavelmente sabem mais do que nós. De qualquer forma, aqui estão elas para garantir.

![](https://github.com/thomasnield/anaconda_python_eda/raw/public/resource/7Od2TS0O.svg)


## Variável Velocidade vs. Altura

A seguir, vamos dar uma olhada em `SPEED`. Quanto mais rápido um avião estiver voando, maior a probabilidade de ser danificado ao colidir com um pássaro, resultando em um relatório de colisão com pássaros. Um pássaro que colide com um avião em movimento lento tem menos probabilidade de ser considerado uma colisão com pássaros se não houver danos, certo? No entanto, um motor em rotação em uma aeronave parada pode sugar um pássaro e certamente também ser considerado uma colisão com pássaros.

Vamos dar uma olhada.

In [ ]:
df["SPEED"].hist(bins=50)

Parece que temos uma distribuição normal aqui, conforme indicado pela curva em forma de sino, com alguns valores extremos à direita. Isso é interessante. Talvez isso seja explicado pela altitude, já que os aviões só atingem suas altas velocidades em altitudes de cruzeiro. Então, o que acontece se somarmos isso à velocidade?

In [ ]:
df.plot.scatter(x="HEIGHT",y="SPEED")


Bem, ao analisarmos colisões com pássaros, certamente vemos que, à medida que a velocidade aumenta, a altura também aumenta. No entanto, ao se aproximar de altitudes de cruzeiro acima de 15.000 pés, observamos uma queda acentuada nos incidentes.

## Variável Distância vs. Altura

Também podemos observar que, quando definimos a distância em relação à altura, observamos que os incidentes de colisão com pássaros se tornam rapidamente congestionados, à medida que ambos se aproximam de 0.

In [ ]:
df.plot.scatter(x="DISTANCE",y="HEIGHT")


Isso está de acordo com nossa descoberta de que colisões com pássaros acontecem durante as fases de aproximação e decolagem do voo.

## Análise de Séries Temporais

Agora, vamos analisar os incidentes com pássaros por semana. Faremos algumas conversões de séries temporais e, em seguida, plotaremos a contagem de incidentes por semana.

In [ ]:
df_series = pd.DataFrame({"INCIDENT_DATE" : df["INCIDENT_DATE"], "STRIKE_COUNT" : 1})
df_series.set_index('INCIDENT_DATE', inplace=True)

df_series \
 .resample("W") \
 .sum() \
 .plot(kind='line', figsize=(15,3), title="Time Series Analysis")


Nossa, temos algo bastante cíclico aqui. Vamos analisar um único ano e identificar alguma sazonalidade.

In [ ]:
df_series \
 .loc["2021"] \
 .resample("W") \
 .sum() \
 .plot(kind='line', figsize=(15,3), title="Time Series Analysis")


Portanto, se isso for representativo do ciclo típico, vemos os incidentes com pássaros aumentarem em abril e, em seguida, aumentarem novamente de forma acentuada após junho. O declínio começa em meados de outubro.

Vamos analisar de 2021 em diante, por semana.

In [ ]:
df_series \
 .loc["2021":] \
 .resample("W") \
 .sum() \
 .plot(kind='line', figsize=(15,3), title="Time Series Analysis")


Os motivos pelos quais vemos um pico no verão e uma queda no inverno podem ter a ver, em grande parte, com os padrões de migração das aves. Na América do Norte, as aves voam para o sul no inverno e para o norte no verão. Também podemos levantar a hipótese de que as viagens de verão trazem mais voos, mas as pessoas também viajam muito em dezembro para férias. Se pesquisássemos esse assunto a fundo, descobriríamos, de acordo com a [FAA](https://www.faa.gov/air_traffic/publications/atpubs/aip_html/part2_enr_section_5.6.html), que "o risco de colisões com pássaros aumenta devido à migração de pássaros durante os meses de março a abril e de agosto a novembro". Aliás, esse é o maior fator no risco de colisões com pássaros, e a série temporal acima demonstra isso!

E quanto ao horário? Isso influencia?

In [ ]:
df_series_tm =  df["TIME"].dropna().dt.components.hours.value_counts().sort_index()
df_series_tm.plot(kind='line', figsize=(15,3), title="Bird Strikes by Hour")


Ok, é bastante evidente que as colisões com pássaros ocorrem durante o dia, com um pico bem antes das 10h. Faz sentido que muitas aves sejam menos ativas à noite. Poderíamos levantar a hipótese de que as colisões noturnas menos frequentes com pássaros sejam aves noturnas, como corujas, mas explore essas áreas com calma.

Provavelmente seria útil analisar essa curva separadamente para cada mês do ano, para que possamos levar em conta a sazonalidade e o aumento ou a diminuição dos incidentes de migração de pássaros.

In [ ]:
by_month_tm = pd.DataFrame({
    "MONTH" : df[df["TIME"].notna()]["INCIDENT_DATE"].dt.month, 
    "HOUR" : df[df["TIME"].notna()]["TIME"].dt.components.hours, 
    "STRIKES" : 1 
}).groupby(["MONTH","HOUR"]) \
.sum() \
.reset_index() \
.pivot(index="HOUR",columns="MONTH",values="STRIKES")

by_month_tm.plot(kind='line', figsize=(15,3), title="Bird Strikes by Hour for Each Month")

As tendências de pico e queda ao longo do dia geralmente se mantêm para cada mês, sendo os meses de verão mais intensos. Curiosamente, setembro (mês 9), outubro (mês 10) e maio (mês 5) apresentam um aumento após as 20h (hora 20). Esses podem ser meses migratórios, então talvez haja mais atividade de aves nas horas mais tardias do dia? É difícil dizer, e há muitas hipóteses que podemos explorar! E só agora sabemos como procurar porque fizemos esta análise de séries temporais.

A análise de séries temporais é mais uma maneira de detectar tendências e padrões em nossos dados nos quais um componente cronológico desempenha um papel. Apenas tome cuidado e sempre pergunte de onde vieram os dados! Por exemplo, se você ingerir todo o conjunto de dados de colisões com aves da FAA (e não apenas após 2015), poderá descobrir que os relatos de colisões com aves aumentaram rapidamente desde 2008. Isso significa que as colisões com aves aumentaram? Não, mas os relatos de colisões com aves aumentaram devido a um maior alcance e relatórios proativos desde o incidente de Sully.

## Que História Vamos Contar?

Tenha em mente que esta é uma análise exploratória de dados resumida para os propósitos desta aula. É altamente recomendável conferir [_Análise Exploratória de Dados com Python_](https://learning.anaconda.cloud/exploratory-data-analysis-eda-with-python) caso você ainda não tenha feito isso. Mas essas descobertas que fizemos aqui devem nos ajudar a começar a contar a história.

Então, qual é a nossa tese central? Acho que seria esta:

> Os relatos de colisões com pássaros ocorrem intensamente em fases de voo de baixa altitude, especialmente na aproximação. Eles também são altamente sazonais devido aos padrões de migração, subindo à medida que o verão se aproxima e diminuindo à medida que o inverno se aproxima. Também são muito mais propensos a ocorrer nas primeiras horas da manhã.

Embora esta possa não ser uma análise muito inovadora para os pilotos (a FAA também concluiu essas descobertas e outras), você fez exatamente o que lhe foi pedido: realizou um estudo independente sobre as causas das colisões com pássaros, e suas descobertas confirmaram as hipóteses atuais.

Reuniríamos os gráficos acima e os usaríamos para embasar nossas descobertas. Falaremos sobre gráficos na próxima seção.

Os pilotos podem pedir que você encontre coisas mais interessantes que outros possam ter perdido. Você deve evitar essa pressão, pois isso é p-hacking, algo que discutiremos mais adiante. Em vez disso, você deve responder com "há algo específico (uma hipótese) que você deseja investigar?". Recoloque neles o ônus de ter uma hipótese, como "os especialistas", em vez de se sentirem pressionados a analisar dados às cegas em busca de correlações. As análises mais produtivas começam com hipóteses, e a mineração de dados, embora apropriada em algumas situações, rapidamente tem um retorno decrescente.

## EXERCÍCIO

Se o seu público não fosse um sindicato de pilotos, mas sim cientistas de dados em uma grande conferência de IA com 300 participantes para sua apresentação, o que você faria para mudar sua narrativa?

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

**Se você estiver apresentando para cientistas de dados em uma conferência de IA em vez de um sindicato de pilotos, aqui estão algumas coisas a considerar:**

* Mantenha as mesmas descobertas, mas use estatísticas mais detalhadas
* Público altamente técnico, mostre o código
* Use um notebook Jupyter em vez do PowerPoint
* Explique o jargão da aviação a eles, já que poucos (se houver) serão pilotos
* Desafie-os a pensar além dos dados